In [94]:
import numpy as np
import pandas as pd
from lightfm.datasets import fetch_movielens
data = fetch_movielens(min_rating=5.0)
from lightfm import LightFM

from developed_methods import *

In [95]:
model = LightFM(loss='warp')
%time model.fit(data['train'], epochs=30, num_threads=2)

Wall time: 1.4 s


In [96]:
dtrain = pd.read_csv("data/train.csv")
dtest = pd.read_csv("data/test.csv")

## mapping 
from sklearn import preprocessing
le_user = preprocessing.LabelEncoder()
le_user.fit(np.append(dtrain['user_id'], dtest["user_id"]))
dtrain['user_id'] = le_user.transform(dtrain["user_id"])
dtest["user_id"] = le_user.transform(dtest["user_id"])

le_item = preprocessing.LabelEncoder()
le_item.fit(np.append(dtrain['item_id'], dtest["item_id"]))
dtrain["item_id"] = le_item.transform(dtrain["item_id"])
dtest["item_id"] = le_item.transform(dtest["item_id"])

In [97]:
# train_pair, train_rating
train_pair = dtrain[['user_id', 'item_id']].values
train_rating = dtrain['rating'].values

# train_pair
test_pair = dtest[['user_id', 'item_id']].values
n_user, n_item = max(train_pair[:,0].max(), test_pair[:,0].max())+1, max(train_pair[:,1].max(), test_pair[:,1].max())+1

In [99]:
from scipy.sparse import lil_matrix

interactions = lil_matrix((n_user, n_item))
for i in range(len(train_pair)):
    user, item = train_pair[i]
    interactions[user, item] = train_rating[i]

In [100]:
model_lightFM = LightFM(no_components = 20)
model_lightFM.fit(interactions, epochs=300, num_threads=4, verbose=True)

Epoch: 100%|██████████| 300/300 [00:19<00:00, 15.69it/s]


In [104]:
pred = model_lightFM.predict(np.array(dtrain["user_id"]), np.array(dtrain["item_id"]))

Extremely high RMSE

In [105]:
def rmse(true, pred):
	return np.sqrt(np.mean((pred - true)**2))

rmse(train_rating, pred)

9.028893319080478